In [1]:
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score  
import pandas as pd
import numpy as np

In [6]:
path = r'G:\Other computers\My Laptop D14\Belgeler\CODING\Python.PROJECTS\MTM-DataMining-FlightDelay\manipulated_data\flightDelay_preprocessed_for_DT.csv'

data = pd.read_csv(path)

df = data.copy()

In [7]:
df.head()

,Flight_Number_Marketing_Airline,AirTime,DepTimeBlk_Encoded,OriginAirport_Encoded,Target
0,1582,93.0,10,15,2
1,1583,93.0,4,100,1
2,1586,178.0,4,196,0
3,1587,55.0,8,15,0
4,1587,67.0,10,231,2


In [4]:
y = df['Target'].values
X = df.drop('Target', axis=1).values

In [5]:
np.set_printoptions(suppress=True)

In [6]:
X #Öznitelikler 

array([[1582.,   93.,   10.,   15.],
       [1583.,   93.,    4.,  100.],
       [1586.,  178.,    4.,  196.],
       ...,
       [5951.,  103.,    4.,  228.],
       [5951.,  102.,    4.,  228.],
       [5951.,  103.,    4.,  228.]])

In [7]:
y #Hedef değişken

array([2, 1, 0, ..., 0, 0, 0], dtype=int64)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23)

In [9]:
model = DecisionTreeClassifier(random_state=23)
model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=23)

In [10]:
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.4117523389046568

Confusion Matrix:
 [[3531 3126 3016]
 [3681 6932 3737]
 [4129 4695 5205]]


### Cross Validation

In [11]:
model = DecisionTreeClassifier(random_state=23)

# Cross-validation işlemi
cv_results = cross_validate(model, X, y, cv=5, scoring='accuracy', return_train_score=True)

# Test seti üzerindeki skorlar
test_scores = cv_results['test_score']
print("Test Seti Accuracy Skorları:", test_scores)
print('Test Seti Accuracy Ortalaması: ',round(test_scores.mean(), 2))

# Eğitim seti üzerindeki skorlar
train_scores = cv_results['train_score']
print("Eğitim Seti Accuracy Skorları:", train_scores)
print('Eğitim Seti Accuracy Ortalaması: ',round(train_scores.mean(), 2))


Test Seti Accuracy Skorları: [0.30400505 0.27312625 0.2286345  0.29685693 0.32979422]
Test Seti Accuracy Ortalaması:  0.29
Eğitim Seti Accuracy Skorları: [0.82231435 0.81407557 0.81952867 0.82719586 0.8204168 ]
Eğitim Seti Accuracy Ortalaması:  0.82


Şuana kadar görülebileceği üzere modelin test performansı oldukça düşüktür. Oysaki Train seti üzerinde hatırı sayılır bir performans sergilediği söylenebilir. Bu durumda modelin **overfitting** yani **aşırı öğrenme** durumu yaşadığı söylenebilir. Bu durumun önüne geçebilmek için, karar ağaçlarında **pruning** yani **budama** işlemi gerçekleştirilmelidir.

Modeli varsayılan parametreler ile kullanmak çok verimli bir yaklaşım sağlamamaktadır. Model optimizasyonu gerçekleştirip yeniden deneyelim.

### Decision Tree Model Tune

Pruning, ağacın aşırı öğrenmeyi önlemek ve genelleme yeteneğini artırmak için gereksiz dallarını kesme işlemidir. Pruning için kullanılan bazı önemli parametreleri anlamak için aşağıdaki açıklamalara göz atalım:

**max_depth**: Ağacın maksimum derinliğini belirtir. Bu, ağacın kaç seviye derinliğe kadar inşa edileceğini kontrol eder. Daha düşük bir max_depth değeri, ağacın daha küçük ve daha basit olmasını sağlar. _Varsayılan değer: None_

**min_samples_split**: Bir iç düğümü bölmek için gereken minimum örnek sayısını belirtir. Düşük değerler, ağacın daha fazla bölünme yapmasına neden olabilir. _Varsayılan değer: 2_

**min_samples_leaf**: Bir yaprağın minimum örnek sayısını belirtir. Düşük değerler, yaprakların daha az örnek içermesine ve böylece daha fazla ayrıntıya inmesine neden olabilir.  _Varsayılan değer: 1_

**max_features**: Her bir bölme noktasında göz önünde bulundurulacak maksimum özellik sayısını belirtir. Daha küçük bir değer seçmek, ağacın daha fazla rastgelelik eklemesini sağlar.  _Varsayılan değer: None_

In [16]:
from sklearn.model_selection import GridSearchCV

# Karar ağacı modelini tanımlayalım
model = DecisionTreeClassifier(random_state=23)

# Parametre aralıklarını belirleyelim
param_grid = {
    'max_depth': [None, 10, 20, 30, 50, 75],
    'min_samples_split': [2, 3, 5, 7, 10],
    'min_samples_leaf': [1, 2, 4, 6, 10],
    'max_features': [None, 'sqrt', 'log2', 2, 3, 4, 5]
}

# GridSearchCV kullanarak en iyi parametreleri bulalım
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid_search.fit(X, y)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=23),
             param_grid={'max_depth': [None, 10, 20, 30, 50, 75],
                         'max_features': [None, 'sqrt', 'log2', 2, 3, 4, 5],
                         'min_samples_leaf': [1, 2, 4, 6, 10],
                         'min_samples_split': [2, 3, 5, 7, 10]},
             return_train_score=True, scoring='accuracy')

In [17]:
print("En iyi parametreler:", grid_search.best_params_)


En iyi parametreler: {'max_depth': 10, 'max_features': 3, 'min_samples_leaf': 6, 'min_samples_split': 2}


In [18]:
tuned_model = grid_search.best_estimator_

cv_results = cross_validate(tuned_model, X, y, cv=5, return_train_score=True)

In [20]:
# Eğitim (train) skorlarını bastıralım
# Test seti üzerindeki skorlar
test_scores = cv_results['test_score']
print("Test Seti Accuracy Skorları:", test_scores)
print('Test Seti Accuracy Ortalaması: ',round(test_scores.mean(), 2))

# Eğitim seti üzerindeki skorlar
train_scores = cv_results['train_score']
print("Eğitim Seti Accuracy Skorları:", train_scores)
print('Eğitim Seti Accuracy Ortalaması: ',round(train_scores.mean(), 2))

Test Seti Accuracy Skorları: [0.35680122 0.31772312 0.31217807 0.35167665 0.37460251]
Test Seti Accuracy Ortalaması:  0.34
Eğitim Seti Accuracy Skorları: [0.47076678 0.47689659 0.47091789 0.47026089 0.46100074]
Eğitim Seti Accuracy Ortalaması:  0.47


Bu şekilde de underfit olayını gözlemlemiş oluyoruz. Her ne kadar model, test setindeki en iyi doğruluk "accuracy" değeri için optimize edilse de. Parametre manipulasyonu sonucunda yeterince öğrenemiyor. 

Decision tree algoritmasını böylece uygulamış olduk, sonuçlar kesinlikle tatmin edici değil. Bu durum algoritmanın zayıflığından mı kaynaklanıyor? Bunu test edebilmek adına farklı algoritmalar denenecektir. Sırasıyla:
- Random Forest
- LGBM
- Catboost
Algoritmaları veri seti üzerinde uygulanacaktır. 

Bu algoritmaların seçilmesinin altında yatan temel sebep, One-Hot Encoding gerektirmiyor olmalarıdır. Böylece boyut laneti ile karşı karşı kalmayacağımızı umuyoruz.


## Model: Random Forest

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [23]:

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

cv_results = cross_validate(rf_classifier, X, y, cv=5, scoring='accuracy', return_train_score=True)

In [24]:
# Eğitim (train) skorlarını bastıralım
test_scores = cv_results['test_score']
print("Test Seti Accuracy Skorları:", test_scores)
print('Test Seti Accuracy Ortalaması: ',round(test_scores.mean(), 2))

# Eğitim seti üzerindeki skorlar
train_scores = cv_results['train_score']
print("Eğitim Seti Accuracy Skorları:", train_scores)
print('Eğitim Seti Accuracy Ortalaması: ',round(train_scores.mean(), 2))

Test Seti Accuracy Skorları: [0.31948386 0.26587302 0.24479659 0.27641123 0.3480329 ]
Test Seti Accuracy Ortalaması:  0.29
Eğitim Seti Accuracy Skorları: [0.82229464 0.81406243 0.8195221  0.82718272 0.82037081]
Eğitim Seti Accuracy Ortalaması:  0.82


In [25]:
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score


scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1': make_scorer(f1_score, average='weighted')
}

cv_results = cross_validate(rf_classifier, X, y, cv=5, scoring=scoring, return_train_score=True)

In [26]:
# Display the results
print("Test Set Accuracy Scores:", cv_results['test_accuracy'])
print('Test Set Accuracy Mean:', round(cv_results['test_accuracy'].mean(), 2))

print("Test Set Precision Scores:", cv_results['test_precision'])
print('Test Set Precision Mean:', round(cv_results['test_precision'].mean(), 2))

print("Test Set Recall Scores:", cv_results['test_recall'])
print('Test Set Recall Mean:', round(cv_results['test_recall'].mean(), 2))

print("Test Set F1 Scores:", cv_results['test_f1'])
print('Test Set F1 Mean:', round(cv_results['test_f1'].mean(), 2))

Test Set Accuracy Scores: [0.31948386 0.26587302 0.24479659 0.27641123 0.3480329 ]
Test Set Accuracy Mean: 0.29
Test Set Precision Scores: [0.33102297 0.23948895 0.24643355 0.27932497 0.37479418]
Test Set Precision Mean: 0.29
Test Set Recall Scores: [0.31948386 0.26587302 0.24479659 0.27641123 0.3480329 ]
Test Set Recall Mean: 0.29
Test Set F1 Scores: [0.2945065  0.23758738 0.24355534 0.27675345 0.34853656]
Test Set F1 Mean: 0.28


Model zaten baya kötü performans gösterdiği için bu metrikler de bir şey ifade etmiyor.

### Random Forest Model Tune

In [28]:
# Karar ağacı modelini tanımlayalım
rf_classifier = RandomForestClassifier(n_estimators=20, random_state=42)

# Parametre aralıklarını belirleyelim
param_grid = {
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 3, 5],
    'min_samples_leaf': [2, 4, 5],
    'max_features': ['sqrt', 'log2', 3, 5]
}

# GridSearchCV kullanarak en iyi parametreleri bulalım
grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid_search.fit(X, y)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(n_estimators=20, random_state=42),
             param_grid={'max_depth': [5, 10, 20],
                         'max_features': ['sqrt', 'log2', 3, 5],
                         'min_samples_leaf': [2, 4, 5],
                         'min_samples_split': [2, 3, 5]},
             return_train_score=True, scoring='accuracy')

In [29]:
print("En iyi parametreler:", grid_search.best_params_)

tuned_model = grid_search.best_estimator_

cv_results = cross_validate(tuned_model, X, y, cv=5, return_train_score=True)

En iyi parametreler: {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'min_samples_split': 2}


In [30]:
# Eğitim (train) skorlarını bastıralım
test_scores = cv_results['test_score']
print("Test Seti Accuracy Skorları:", test_scores)
print('Test Seti Accuracy Ortalaması: ',round(test_scores.mean(), 2))

# Eğitim seti üzerindeki skorlar
train_scores = cv_results['train_score']
print("Eğitim Seti Accuracy Skorları:", train_scores)
print('Eğitim Seti Accuracy Ortalaması: ',round(train_scores.mean(), 2))

Test Seti Accuracy Skorları: [0.35228109 0.32247976 0.35251761 0.32539683 0.38474679]
Test Seti Accuracy Ortalaması:  0.35
Eğitim Seti Accuracy Skorları: [0.43091973 0.43695756 0.44344215 0.44315964 0.43533848]
Eğitim Seti Accuracy Ortalaması:  0.44


Son olarak bir de kendimiz bir model türetmeyi deneyelim.

In [32]:
my_RF_model = RandomForestClassifier(
    n_estimators=25, 
    random_state=42,
    max_depth= 20,
    max_features='sqrt',
    min_samples_leaf = 6,
    min_samples_split = 2
)

cv_results = cross_validate(my_RF_model, X, y, cv=5, return_train_score=True)

# Eğitim (train) skorlarını bastıralım
test_scores = cv_results['test_score']
print("Test Seti Accuracy Skorları:", test_scores)
print('Test Seti Accuracy Ortalaması: ',round(test_scores.mean(), 2))

# Eğitim seti üzerindeki skorlar
train_scores = cv_results['train_score']
print("Eğitim Seti Accuracy Skorları:", train_scores)
print('Eğitim Seti Accuracy Ortalaması: ',round(train_scores.mean(), 2))

Test Seti Accuracy Skorları: [0.34071796 0.27115526 0.27365184 0.29054977 0.3365746 ]
Test Seti Accuracy Ortalaması:  0.3
Eğitim Seti Accuracy Skorları: [0.62395291 0.61955101 0.62100955 0.61917652 0.61408073]
Eğitim Seti Accuracy Ortalaması:  0.62


Ağaç temelli boosting algoritmaları da denenebilir, lakin modeller random üretilecek sonuçlardan bile kötü çalışmaktadır. Belli ki bir şeyler ters gidiyor. İyisi mi Naive Bayes deneyelim.

In [8]:
df.shape


(190259, 5)

In [9]:
originAirport_counts = df['OriginAirport_Encoded'].value_counts()
originAirport_counts

OriginAirport_Encoded
15     11046
75     10860
57      8860
207     8384
74      7197
       ...  
175       13
13        11
149        4
63         3
221        2
Name: count, Length: 293, dtype: int64

In [10]:
selected_list = originAirport_counts[originAirport_counts > 1000].index

df2 = df[df['OriginAirport_Encoded'].isin(selected_list)]

In [11]:
df2.shape

(142723, 5)

In [12]:
flightNumber_counts = df2['Flight_Number_Marketing_Airline'].value_counts()
flightNumber_counts

Flight_Number_Marketing_Airline
2438    182
493     165
2368    147
1129    143
2377    143
       ... 
5319      3
4688      2
3751      2
5080      1
6196      1
Name: count, Length: 2460, dtype: int64

In [13]:
selected_list = flightNumber_counts[flightNumber_counts > 100].index

df3 = df2[df2['Flight_Number_Marketing_Airline'].isin(selected_list)]

In [14]:
df3.shape

(14104, 5)

In [15]:
df3.reset_index(drop=True, inplace=True)

In [16]:
df3

,Flight_Number_Marketing_Airline,AirTime,DepTimeBlk_Encoded,OriginAirport_Encoded,Target
0,1588,39.0,1,33,1
1,1620,58.0,4,174,0
2,1623,92.0,9,15,0
3,1642,154.0,1,81,2
4,1642,144.0,5,100,2
...,...,...,...,...,...
14099,4922,78.0,6,74,0
14100,4922,71.0,6,74,0
14101,4922,78.0,6,74,0
14102,4922,80.0,6,74,0


Düşük sınıflı değişkenlerden kurtulmuş olmamız gerekirdi. 

In [17]:
X_ = df3.drop('Target', axis=1).values
y_ = df3['Target'].values

In [ ]:
# En iyi parametreler: {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'min_samples_split': 2}

In [60]:
modelRF_v2 = RandomForestClassifier(
    n_estimators=25,
    random_state=42,
    max_depth= 20,
    max_features='sqrt',
    min_samples_leaf=3,
    min_samples_split=2
)

cv_results = cross_validate(modelRF_v2, X_, y_, cv=5, return_train_score=True)

In [61]:
# Eğitim (train) skorlarını bastıralım
test_scores = cv_results['test_score']
print("Test Seti Accuracy Skorları:", test_scores)
print('Test Seti Accuracy Ortalaması: ',round(test_scores.mean(), 2))

# Eğitim seti üzerindeki skorlar
train_scores = cv_results['train_score']
print("Eğitim Seti Accuracy Skorları:", train_scores)
print('Eğitim Seti Accuracy Ortalaması: ',round(train_scores.mean(), 2))

Test Seti Accuracy Skorları: [0.44062389 0.37575328 0.23431407 0.26019142 0.33510638]
Test Seti Accuracy Ortalaması:  0.33
Eğitim Seti Accuracy Skorları: [0.6787202  0.6787202  0.68004963 0.69440752 0.67874867]
Eğitim Seti Accuracy Ortalaması:  0.68


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_, y_, random_state=30, train_size=0.2)

modelRF_v2 = RandomForestClassifier(
    n_estimators=20,
    random_state=30,
    max_depth= 20,
    max_features='sqrt',
    min_samples_leaf=3,
    min_samples_split=2
)

modelRF_v2.fit(X_train, y_train)

y_pred = modelRF_v2.predict(X_test)

In [21]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Test Set Accuracy Scores:", accuracy)

# Calculate precision
precision_scores = precision_score(y_test, y_pred, average=None)
print("Test Set Precision Scores:", precision_scores)
print("Test Set Precision Mean:", precision_scores.mean())

# Calculate recall
recall_scores = recall_score(y_test, y_pred, average=None)
print("Test Set Recall Scores:", recall_scores)
print("Test Set Recall Mean:", recall_scores.mean())

# Calculate F1 score
f1_scores = f1_score(y_test, y_pred, average=None)
print("Test Set F1 Scores:", f1_scores)
print("Test Set F1 Mean:", f1_scores.mean())

Test Set Accuracy Scores: 0.44080113434952145
Test Set Precision Scores: [0.34267134 0.46461187 0.46030099]
Test Set Precision Mean: 0.42252806460084535
Test Set Recall Scores: [0.24273565 0.43160127 0.5673774 ]
Test Set Recall Mean: 0.41390477324380154
Test Set F1 Scores: [0.28417341 0.44749863 0.50826091]
Test Set F1 Mean: 0.4133109815709584


## Model: LGBM

In [22]:
import lightgbm as lgb

X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.2, random_state=42)


lgb_classifier = lgb.LGBMClassifier(n_estimators=50, random_state=30)
lgb_classifier.fit(X_train, y_train)

y_pred = lgb_classifier.predict(X_test)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000321 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 437
[LightGBM] [Info] Number of data points in the train set: 11283, number of used features: 4
[LightGBM] [Info] Start training from score -1.372825
[LightGBM] [Info] Start training from score -1.108230
[LightGBM] [Info] Start training from score -0.875947


In [23]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Test Set Accuracy Scores:", accuracy)

# Calculate precision
precision_scores = precision_score(y_test, y_pred, average=None)
print("Test Set Precision Scores:", precision_scores)
print("Test Set Precision Mean:", precision_scores.mean())

# Calculate recall
recall_scores = recall_score(y_test, y_pred, average=None)
print("Test Set Recall Scores:", recall_scores)
print("Test Set Recall Mean:", recall_scores.mean())

# Calculate F1 score
f1_scores = f1_score(y_test, y_pred, average=None)
print("Test Set F1 Scores:", f1_scores)
print("Test Set F1 Mean:", f1_scores.mean())

Test Set Accuracy Scores: 0.4835164835164835
Test Set Precision Scores: [0.45517241 0.50598086 0.47728614]
Test Set Precision Mean: 0.47947980357677933
Test Set Recall Scores: [0.19130435 0.44479495 0.68559322]
Test Set Recall Mean: 0.440564173615486
Test Set F1 Scores: [0.26938776 0.47341914 0.56278261]
Test Set F1 Mean: 0.4351965006727247


## Model: CatBoost

In [24]:
from catboost import CatBoostClassifier

# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.2, random_state=42)

# CatBoost modelini oluşturma ve eğitme
catboost_classifier = CatBoostClassifier(iterations=50, random_state=42, verbose=False)
catboost_classifier.fit(X_train, y_train)

# Test seti üzerinde tahmin yapma
y_pred = catboost_classifier.predict(X_test)

# Modelin performansını değerlendirme
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.48635235732009924


In [25]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Test Set Accuracy Scores:", accuracy)

# Calculate precision
precision_scores = precision_score(y_test, y_pred, average=None)
print("Test Set Precision Scores:", precision_scores)
print("Test Set Precision Mean:", precision_scores.mean())

# Calculate recall
recall_scores = recall_score(y_test, y_pred, average=None)
print("Test Set Recall Scores:", recall_scores)
print("Test Set Recall Mean:", recall_scores.mean())

# Calculate F1 score
f1_scores = f1_score(y_test, y_pred, average=None)
print("Test Set F1 Scores:", f1_scores)
print("Test Set F1 Mean:", f1_scores.mean())

Test Set Accuracy Scores: 0.48635235732009924
Test Set Precision Scores: [0.4548495  0.50337079 0.48284314]
Test Set Precision Mean: 0.4803544740331717
Test Set Recall Scores: [0.19710145 0.47108307 0.66779661]
Test Set Recall Mean: 0.44532704329900313
Test Set F1 Scores: [0.27502528 0.48669202 0.56045519]
Test Set F1 Mean: 0.4407241617673033
